In [305]:
import requests
import shutil
from lxml import html
from bs4 import BeautifulSoup as bs
from IPython.display import Image, display

In [85]:
sport_dict = {
         '田徑'   : 'A',
         '足球'   : 'B',
         '籃球'   : 'C',
         '排球'   : 'D',
         '網球'   : 'E',
         '棒球'   : 'F',
         '壘球'   : 'G',
         '橄欖球' : 'H',
         '羽球'   : 'S',
         '桌球'   : 'T'
    }
volley_court_dict = {
     '00':'光復低場排球1',
     '01':'光復低場排球1',
     '02':'光復低場排球1',
     '10':'光復低場排球2',
     '11':'光復低場排球2',
     '12':'光復低場排球2',
     '20':'光復低場排球3',
     '21':'光復低場排球3',
     '22':'光復低場排球3',
     '30':'光復低場排球4',
     '31':'光復低場排球4',
     '32':'光復低場排球4',
     '40':'光復低場排球5',
     '41':'光復低場排球5',
     '42':'光復低場排球5',
     '50':'光復高場排球東',
     '51':'光復高場排球東',
     '52':'光復高場排球東',
     '60':'光復高場排球西',
     '61':'光復高場排球西',
     '62':'光復高場排球西',
}
time_dict = {
     '0':'0600~1200',
     '1':'1200~1800',
     '2':'1800~2100'
}

In [49]:
def get_sport_raw(date:str, sport_in:str):
    session = requests.session()
    url = 'https://cet.acad.ncku.edu.tw/ste/index.php?c=ste11211'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
        # 'Cookie': 'PHPSESSID=1aea228467450d51917fc7f8f835ff86',
        'Cookie': 'PHPSESSID=8f4bace3465bd821988e2ed78be708b8'
        # TODO: 把這邊的 Cookie 改成 CookieJar
        # 'X-Requested-With': 'XMLHttpRequest'
    }

    payload = {
        'sport': sport_dict[sport_in],
        'sdate': date,
        'c': 'ste11211',
        'm': 'read'
    }


    volleyball = session.post(url=url, headers=headers, data=payload)
    # print(session.cookies.get)
    # print(login.cookies, login.cookies.get_dict())
    # print(headers['Cookie'])

    # if login.cookies.get_dict():        #保持cookie有效
        # login.cookies.update(login.cookies)
    # volleyball = session.post(url=url_get, headers=headers, data=payload)
    # print(login.cookies, bool(login.cookies.get_dict()))
    # print(login.text)
    # print(volleyball.text)
    # with open('test.html', 'w', encoding='utf-8') as fp:
    #     fp.write(volleyball.text)

    # replace is for '體育室 <br/>(數學系)' (in raw)
    # but in '體育室 <br>(數學系)' (in html)
    return bs(volleyball.text.replace('<br>', ''), 'html.parser')

In [159]:
# get_sport_raw('20201222', '排球')

In [65]:
def retreive_form(soup):
    if not isinstance(soup, type(bs())):
        raise TypeError("soup must be a " + repr(type(bs())))
    else:
        form =[]
        row = []
        counter = 0
        for content in soup.find_all('td'):
            # print(content, '\t\t', repr(content.string))
            if content.string not in ('\xa0\xa0', '可借用', '不可借用', '已借用'):
                if counter == 3:
                    counter = 0
                    form.append(row)
                    row = []
                row.append(content.string)
                counter += 1
    
    return form

In [43]:
from datetime import datetime, timedelta
def gen_dates(start:str, end:str):
    start_date = datetime.strptime(start, '%Y%m%d')
    end_date = datetime.strptime(end, '%Y%m%d')
    serial = []
    if (end_date - start_date).days >= 0:
        for i in range((end_date - start_date).days+1):
            tmp = start_date + timedelta(days=i)
            serial.append('{0:>{3}4}{1:>{3}2}{2:>{3}2}'.format(tmp.year, tmp.month, tmp.day, 0))
            
    return serial

In [8]:
# output to html file
with open('test1.html', 'w', encoding='utf-8') as fp:
    fp.write(soup.prettify())

NameError: name 'soup' is not defined

In [51]:
new_s = get_sport_raw('20201221', '排球')
t = retreive_form(new_s)
t
# print(t)
# print(type(new_s))
# new_s = new_s.replace('<br/', '')
# print(repr(new_s))

[['F7資訊系', '體育室  ', 'I6物治系'],
 ['H1會計系', '體育室  ', 'B2外文系'],
 ['F1系統系', '體育室  ', '體育室  (數學系)'],
 [None, '體育室  ', 'I5醫學系'],
 ['F2都計系', '體育室  ', 'K5台文所'],
 ['F6測量系', '體育室  ', '體育室  (貴重儀器中心)'],
 ['P8醫工所', '體育室  ', 'I8藥學系']]

In [155]:
seq = gen_dates('20201224', '20201231')
def get_mapped_form(time_seq:list):
    situation = {}
    for dates in time_seq:
        print(dates)
        m = get_sport_raw(dates, '排球')
        situation[dates] = retreive_form(m)
        # print(situation)

In [147]:
# find empty time
def find_free_time(seq:list, gender:str='a'):
    # gender = 'a' -> both gender
    free = []
    for dates in seq:
        for row in range(7):
            for column in range(3):
                # print(type(situation[dates][row][column]))
                if isinstance(situation[dates][row][column], type(None)):
                    element = '{0} {1} {2}'.format(dates, time_dict[str(column)], court_dict[str(row*10+column).zfill(2)])
                    if gender.lower() == 'b': # boy
                        if row in [3, 4, 5]:
                            continue
                    elif gender.lower() == 'g': # girl
                        if row not in [3, 4, 5]:
                            continue
                    free.append(element)
    return free
                    

In [153]:
# format outcome
free = find_free_time(seq, 'g')
def format_free_time(free:list):
    new_free = []
    if len(free) != 1:
        old = -1
        for lines in free:
            if old == -1:
                old = int(lines[4:8])
            elif old != int(lines[4:8]):
                old = int(lines[4:8])
                new_free.append('-' * (len(lines)+6))
            new_free.append(lines)
    
    return new_free

In [154]:
for content in format_free_time(free):
    print(content)

20201228 0600~1200 光復低場排球4
20201228 0600~1200 光復低場排球5
20201228 0600~1200 光復高場排球東
--------------------------------
20201229 0600~1200 光復低場排球4
20201229 0600~1200 光復低場排球5
20201229 0600~1200 光復高場排球東
--------------------------------
20201230 0600~1200 光復低場排球4
20201230 0600~1200 光復低場排球5
20201230 1800~2100 光復低場排球5
20201230 0600~1200 光復高場排球東
--------------------------------
20201231 0600~1200 光復低場排球4
20201231 1800~2100 光復低場排球4
20201231 0600~1200 光復低場排球5
20201231 0600~1200 光復高場排球東


In [256]:
soup = bs(result.text, 'html.parser')
# soup

In [405]:
class a:
    value = 3

In [407]:
def add(t):
    print('before =', t.value)
    t.value = 0
    print('after =', t.value)
    return t 

In [285]:
# display vcode
def get_vcode():
    session = requests.session()
    print(session)
    verify_url = 'https://cet.acad.ncku.edu.tw/ste/index.php?c=verifycode'
    response = session.get(url=verify_url, stream=True, verify=False)
    f = open('check.png', 'wb')
    shutil.copyfileobj(response.raw, f)
    f.close()

    display(Image('./check.png'))

In [279]:
# set cookie

cookie_dict = session.cookies.get_dict()
headers['Cookie'] = cookie_dict['PHPSESSID']

In [281]:
headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
 'Cookie': 'd0a54734ae508c7328c88efb5c6ee78c'}

In [289]:
def login(user_id, passwd, vcode):
    # vcode = input('輸入驗證碼：')
    url = 'https://cet.acad.ncku.edu.tw/ste/index.php?c=auth&m=login'
    print(vcode)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    }

    payload = {
        'user_id': user_id,
        'passwd': passwd,
        'code': '',
        'x': '22',
        'y': '16'
    }
    payload['code'] = vcode

    print(session)
    a = session.post(url=url, headers=headers, data=payload)
    print(a)

In [277]:
headers.keys()
c = '3c33cf6cc15e794e2e82d5db1e95d4da'

dict_keys(['User-Agent'])

In [278]:
a.text.find('排球')

2950

In [290]:
userID = input('USER ID')
passwd = input('PASSWD')
get_vcode()
v = input('VCODE')

In [291]:
login(userID, passwd, v)
a.text

2811
https://cet.acad.ncku.edu.tw/ste/index.php?c=auth&m=login&user_id=F74076077&passwd=Test1029384756&code=2811&x=43&y=23
<Response [200]>


'<html>\n\t<head>\n\t\t<title>成功大學運動場地線上借場系統</title>\n\t\t<LINK rel="shortcut icon" href="images_icon/openwebmail.ico">\n\t\t<meta http-equiv="Content-type" content="text/html; charset=utf8" />\n\t\t<meta name="robots" content="index,follow,archive" />\n\t\t<link href="css/ste_style.css" rel="stylesheet" type="text/css">\n\t\t<link href="css/mis_grid.css" rel="stylesheet" type="text/css">\n\t\t\t\t\n\t\t<link href="/nf/commons/v1.1.3/css/ncku-theme/jquery-ui-1.8.20.custom.css" rel="stylesheet" type="text/css" /> \n\t\t<script src="/nf/commons/v1.1.3/js/jquery.1.7.0.min.js" type="text/javascript"></script>\n\t\t<script src="/nf/commons/v1.1.3/js/jquery-ui-1.8.20.custom.min.js" type="text/javascript"></script>\n\t\t<script src="/nf/commons/v1.1.3/js/jquery.ui.datepicker-zh-TW_utf8.js" type="text/javascript"></script>\n\t\t<script src="/nf/commons/v1.1.3/js/jquery.ztree.core-3.1.min.js" type="text/javascript"></script>\n\t\t<script src="/nf/commons/v1.1.3/js/jquery.nyroModal-1.6.1.min.js"

In [292]:
cookie_dict = session.cookies.get_dict()
headers['Cookie'] = cookie_dict['PHPSESSID']

In [293]:
# stay here
url = 'https://cet.acad.ncku.edu.tw/ste/index.php?c=ste1121'
session.get(url=url, headers=headers)
print(a)

<Response [200]>


In [294]:
a.text

'<html>\n\t<head>\n\t\t<title>成功大學運動場地線上借場系統</title>\n\t\t<LINK rel="shortcut icon" href="images_icon/openwebmail.ico">\n\t\t<meta http-equiv="Content-type" content="text/html; charset=utf8" />\n\t\t<meta name="robots" content="index,follow,archive" />\n\t\t<link href="css/ste_style.css" rel="stylesheet" type="text/css">\n\t\t<link href="css/mis_grid.css" rel="stylesheet" type="text/css">\n\t\t\t\t\n\t\t<link href="/nf/commons/v1.1.3/css/ncku-theme/jquery-ui-1.8.20.custom.css" rel="stylesheet" type="text/css" /> \n\t\t<script src="/nf/commons/v1.1.3/js/jquery.1.7.0.min.js" type="text/javascript"></script>\n\t\t<script src="/nf/commons/v1.1.3/js/jquery-ui-1.8.20.custom.min.js" type="text/javascript"></script>\n\t\t<script src="/nf/commons/v1.1.3/js/jquery.ui.datepicker-zh-TW_utf8.js" type="text/javascript"></script>\n\t\t<script src="/nf/commons/v1.1.3/js/jquery.ztree.core-3.1.min.js" type="text/javascript"></script>\n\t\t<script src="/nf/commons/v1.1.3/js/jquery.nyroModal-1.6.1.min.js"

In [308]:
# get verify code
def get_verify_code():
    verify_url = 'https://cet.acad.ncku.edu.tw/ste/index.php?c=verifycode'
    response = requests.get(url=verify_url, stream=True, verify=False)
    f = open('check.png', 'wb')
    shutil.copyfileobj(response.raw, f)
    f.close()

get_verify_code()
display(Image('./check.png'))

In [309]:
from PIL import Image
import pytesseract
img = Image.open('check.png')
text = pytesseract.image_to_string(img, lang='eng')
print(text[0:4])

2325
